# Ituran Second Response charging points SoH esitmation

In this notebook we will handle the processed time series data to compute the SoH from the charging points (like we did with Watea).  
This would corresponds to the result/soh_estimation step in our pipeline.  

## Setup
Please run the `ituran_second_response_tss_EDA.ipynb` notebook before running this one.  

In [ ]:
! mkdir -p data_cache/
! mkdir -p data_cache/plots

### Import

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import qualitative
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer, StandardScaler

from core.singleton_s3_bucket import bucket
from core.pandas_utils import *
from core.plt_utils import plt_3d_df

### Data Extraction

We will simply load the processed time series data computed in the previous notebook.

In [ ]:
tss = bucket.read_parquet_df("processed_ts/ituran/ituran_tss.parquet")

In [ ]:
tss.dtypes

## SoH estimation  
To estimate the SoH, we will use the charging points' energy_added.  
In this context a charging point is an statistical description of a period of charging from one soc to the next.  
We consider the energy added to be the energy that the battery required to gain that soc.  
We will estimate/express the SoH of the battery as the energy required by the battery to gain a certain soc divided by the energy required by a 100%SoH battery to gain the same soc.  
Since the added energy that a battery requires to gain a certain soc depends on multiple factors, we will try to capture as many factors as possible.  
To estimate the SoH all we really need to estimate is the energy required by a 100%SoH battery to gain a certain soc based on the charging point's factors.  
Then we just need to divide the energy required by the battery to gain a certain soc by the estimated energy required by a 100%SoH battery to gain the same soc in the same conditions.  

### Charging points SoH estimation

#### Data extraction

In [ ]:
def compute_first_charge_soc(tss:DF) -> DF:
    tss["first_charge_soc"] = (
        tss
        .groupby(["vehicle_id", "trimmed_in_charge_idx"])
        ["soc"]
        .transform("first")
    )
    tss["first_charge_soc"] = tss["first_charge_soc"].where(tss["trimmed_in_charge"], pd.NA)
    return tss

CHARGING_POINT_QUANTIZATION = 3

def floor_soc(tss:DF) -> DF:
    return (
        tss
        .assign(floored_soc=floor_to(tss["soc"], CHARGING_POINT_QUANTIZATION))
    )

charging_points:DF = (
    tss
    .pipe(compute_first_charge_soc)
    .pipe(floor_soc)
    .query("vehicle_model == 'geometry c' & trimmed_in_charge")
    .groupby(["vehicle_id", "trimmed_in_charge_idx", "floored_soc"], as_index=False, observed=True)
    .agg(
        energy_added_at_start=pd.NamedAgg(column="cum_energy_added", aggfunc="first"),
        energy_added_at_end=pd.NamedAgg(column="cum_energy_added", aggfunc="last"),
        energy_added=pd.NamedAgg(column="cum_energy_added", aggfunc=series_start_end_diff),
        ac_mode_mean=pd.NamedAgg(column="charging_ac_mode", aggfunc="mean"),
        dc_mode_mean=pd.NamedAgg(column="charging_dc_mode", aggfunc="mean"),
        current=pd.NamedAgg(column="ffilled_charging_current", aggfunc="median"),
        voltage=pd.NamedAgg(column="ffilled_charging_voltage", aggfunc="median"),
        estimated_range=pd.NamedAgg(column="ffilled_estimated_range", aggfunc="median"),
        time_remaining_for_charge=pd.NamedAgg(column="ffilled_time_remaining_for_charge", aggfunc="median"),
        model=pd.NamedAgg(column="vehicle_model", aggfunc="first"),
        first_charge_soc=pd.NamedAgg(column="first_charge_soc", aggfunc="first"),
        duration=pd.NamedAgg(column="date", aggfunc=series_start_end_diff),
        date=pd.NamedAgg(column="date", aggfunc="first"),
        nb_points=pd.NamedAgg(column="soc", aggfunc="size"),
        odometer=pd.NamedAgg(column="odometer", aggfunc="first"),
        original_vehicle_id=pd.NamedAgg("original_vehicle_id", "first")
    )
    .rename(columns={"floored_soc": "soc"})
    .eval("energy_added=energy_added_at_end - energy_added_at_start")
    .eval("soc_added = soc - first_charge_soc")
    .eval("power = current * voltage")
    .eval("in_ac = ac_mode_mean > 0.3")
    .eval("in_dc = dc_mode_mean > 0.3")
    .eval("power = current * voltage")
    .eval("sec_duration = duration.dt.total_seconds()")
    .eval("energy_added_per_point = energy_added / nb_points")
)

sanity_check(charging_points)

#### Charging points EDA

In [ ]:
charging_points["vehicle_id"].value_counts(sort=True)

In [ ]:
plt_3d_df(
    charging_points.query("vehicle_id == 27 & energy_added > 13e3"),
    x="date",
    y="duration",
    z="energy_added",
    color="nb_points",
    log_z=True,
)

In [ ]:
plt_3d_df(
    (
        charging_points
        .query("energy_added.between(26e3, 245e3)")
        .query("in_dc")
        #.query("nb_points == 4")
        #.query("nb_points.between(3, 6)")
        .astype({"vehicle_id": "category"})
    ),
    x='power',
    y="soc",
    z="energy_added",
    color="vehicle_id",
    opacity=0.25,
    size=3,
    width=1500,
    height=1000,
    log_z=True,
    #log_y=True,
    #symbol="in_dc",
)

In [ ]:
charging_points_to_plot = (
    charging_points
    # .query("energy_added.between(26e3, 100e3)")
    .query("in_dc")
    # .query("vehicle_id == 27")
)

# Initialize 3D plot
fig = go.Figure()

x='power'
Y='soc'
Z='energy_added'

# Add a trace for each `trimmed_in_charge_idx` group
for (vehicle_id, trimmed_in_charge_idx), group_data in charging_points_to_plot.groupby(["vehicle_id", "trimmed_in_charge_idx"]):
    fig.add_trace(go.Scatter3d(
        x=group_data[x],
        y=group_data[Y],
        z=group_data[Z],
        mode='lines',
        name=f'Group {trimmed_in_charge_idx} (vehicle_id: {vehicle_id})'
    ))

# Update layout
fig.update_layout(
    title="Charges time series",
    scene=dict(
        xaxis_title=x,
        yaxis_title=Y,
        zaxis_title=Z,
        zaxis=dict(
            type='log'
        ),
        camera=dict(
            projection=dict(
                type='orthographic'
            )
        )
    ),
    width=1500,
    height=1000,
    showlegend=True
)

# Show the plot
fig.show()

In [ ]:
# Initialize 3D plot
fig = go.Figure()

x='soc'
Y='nb_points'
Z='energy_added'

charging_points_to_plot = (
    charging_points
    # .query("energy_added.between(26e3, 100e3)")
    .query("in_dc")
    .dropna(subset=[x, Y, Z], how="any")
    .query("energy_added > 1e5")
    .query("nb_points < 30")
)

# Add a trace for each `trimmed_in_charge_idx` group
for (vehicle_id, trimmed_in_charge_idx), group_data in charging_points_to_plot.groupby(["vehicle_id", "trimmed_in_charge_idx"]):
    fig.add_trace(go.Scatter3d(
        x=group_data[x],
        y=group_data[Y],
        z=group_data[Z],
        mode='lines',
        name=f'Group {trimmed_in_charge_idx} (vehicle_id: {vehicle_id})'
    ))

# Update layout
fig.update_layout(
    title="Charges time series",
    scene=dict(
        xaxis_title=x,
        yaxis_title=Y,
        zaxis_title=Z,
        zaxis=dict(
            type='log'
        ),
        camera=dict(
            projection=dict(
                type='orthographic'
            )
        )
    ),
    width=1500,
    height=1000,
    showlegend=True
)

# Show the plot
fig.show()

## Default Energy added fitting
We will try to fit a model to estimate the default(brand new vehicle) energy added required for a charging point to complete based on its power, soc and nb_points.  
And them, express the SoH as the ratio between the energy that charging point actually required and this estimated default required energy added.

Let's find the vehicles with the least amount distance traveled to use as a reference point for the default energy added.

In [ ]:
# Note: There is only one odo value per vehicle (>_<)
vehicle_od = (
    charging_points
    .groupby("vehicle_id", observed=True, as_index=False)["odometer"]
    .agg("min")
    .sort_values("odometer")
)
display(vehicle_od)
px.bar(vehicle_od.astype({"vehicle_id": "string"}), x="vehicle_id", y="odometer")

This actually seems like a pretty good distribution of odometers for that small of a sample size. :thumbsup:

We express the SoH of the charging points as their energy added divided by an estimation of the default energy added.  
The default energy added is estimated as the polyfit of the charging points + the residual from that fit to the points of the vehicles with the lowest odometer.  
Then we perform aggregations to get the SoH per vehicle.   

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, cross_val_score

# Define the features and target
FEATURES = ["power", "soc", "nb_points", "voltage", "current", "soc_added"]
TARGETS = ["energy_added"]

# Drop NaN values if any
if charging_points[FEATURES + TARGETS].isna().any().any():
    charging_points = charging_points.dropna(subset=FEATURES + TARGETS, how="any").copy()

# Extract the feature matrix (X) and target vector (y)
X = charging_points[FEATURES].values
y = charging_points[TARGETS].values.ravel()  # Flatten y to 1D for compatibility

# Define the pipeline
pipeline = Pipeline([
    ('standard_scaler', StandardScaler()),  # Standardization
    ('polynomial_features', PolynomialFeatures()),  # Polynomial expansion
    ('linear_regression', LinearRegression())  # Linear regression
])

# Define the hyperparameter grid (degrees 1 to 6)
param_grid = {'polynomial_features__degree': np.arange(1, 11)}

# Perform GridSearchCV with MSE as the scoring metric
grid_search = GridSearchCV(
    pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1
)

# Fit GridSearchCV to find the best polynomial degree
grid_search.fit(X, y)

# Get the best polynomial degree
best_degree = grid_search.best_params_['polynomial_features__degree']
best_mse = -grid_search.best_score_  # Convert back to positive MSE

# Extract all degrees and corresponding MSE scores
degree_mse_pairs = list(zip(
    grid_search.cv_results_['param_polynomial_features__degree'].data,  # Degrees
    -grid_search.cv_results_['mean_test_score']  # Convert back to positive MSE
))

# Print the results
for degree, mse in degree_mse_pairs:
    print(f"Degree: {degree}, MSE: {mse:.4f}")


In [ ]:
def estimate_soh(charging_points:DF, model_degrees:int, soh_col_name:str="soh") -> DF:
    # Define the features and target
    FEATURES = ["power", "soc", "nb_points", "voltage", "current", "soc_added"]
    TARGETS = ["energy_added"]

    if charging_points[FEATURES + TARGETS].isna().any().any():
        charging_points = charging_points.dropna(subset=FEATURES + TARGETS, how="any").copy()

    # Extract the feature matrix (X) and target vector (y)
    x = charging_points[FEATURES].values
    y = charging_points[TARGETS].values

    # Create a pipeline with an exponential transformation step
    model = Pipeline([
        ('standard_scaler', StandardScaler()),  # Standardization
        ('polynomial_features', PolynomialFeatures(model_degrees)),  # Polynomial expansion
        ('linear_regression', LinearRegression())  # Linear regression
    ])

    # Fit the model
    model.fit(x, y)

    charging_points["fit_energy_added"] = model.predict(x)
    charging_points["residual"] = charging_points.eval("fit_energy_added - energy_added")
    charging_points["is_default_soh"] = charging_points["odometer"] < 5000
    default_soh_residual = charging_points.query("is_default_soh")["residual"].mean()
    charging_points["default_energy_added"] = charging_points["fit_energy_added"] + default_soh_residual
    charging_points[soh_col_name] = charging_points.eval("energy_added / default_energy_added")

    return charging_points

In [ ]:
charging_points = (
    charging_points
    .pipe(estimate_soh, 1, "soh_1")
    .pipe(estimate_soh, 2, "soh_2")
    .pipe(estimate_soh, 3, "soh_3")
    .pipe(estimate_soh, 4, "soh_4")
    .pipe(estimate_soh, 5, "soh_5")
    .pipe(estimate_soh, 6, "soh_6")
    .pipe(estimate_soh, 10, "soh_10")
)

### Estimation evaluation

In [ ]:
SOH_COLS = [
    "soh_1",
    "soh_2",
    "soh_3",
    # "soh_4",
    # "soh_5",
    # "soh_6",
    # "soh_10",    
]

VALS_TO_PLT = [soh_col + "_median" for soh_col in SOH_COLS] + [soh_col + "_mean" for soh_col in SOH_COLS]

charging_points_grp = charging_points.query("energy_added > 0").groupby(["vehicle_id", "trimmed_in_charge_idx"], observed=True)
soh_per_charge = charging_points_grp.agg(odometer=pd.NamedAgg("odometer", "last"))
for soh_col in SOH_COLS:
    soh_per_charge[f"{soh_col}_median"] = charging_points_grp[soh_col].median()
    soh_per_charge[f"{soh_col}_mean"] = charging_points_grp[soh_col].mean()

soh_per_charge = soh_per_charge.reset_index()

px.violin(
    soh_per_charge.melt(["vehicle_id", "odometer"], VALS_TO_PLT).astype({"vehicle_id": "category"}),
    x="odometer",
    y="value",
    facet_row="variable",
    color="vehicle_id",
    height=1000,
    points="all",
).update_yaxes(matches=None)

In [ ]:
soh_per_vin = (
    charging_points
    .groupby("vehicle_id", observed=True, as_index=False)
    .agg({
            "soh_1":  "median",
            "soh_2":  "median",
            "soh_3":  "median",
            "soh_4":  "median",
            "soh_5":  "median",
            "soh_6":  "median",
            "soh_10": "median",
            "odometer": "last",
    })
    .melt(
        ["odometer", "vehicle_id"],
        [
            "soh_1",
            "soh_6",
            "soh_2",
            "soh_3",
            "soh_4",
            "soh_5",
            "soh_6",
            "soh_10",
        ],
    )
)
px.scatter(
    soh_per_vin.astype({"vehicle_id": "category"}),
    x="odometer",
    y="value",
    facet_col="variable",
    color="vehicle_id",
    height=750,
)

We can see that going from 1 to 10 degrees polynomial representation of the default energy added, we end up with completly different results.  
I believe that we could get away with a simple hyper parameter tunning based on cross validation to find the right number of params.  
We will arbitrarely choose the one computed with 6 degree polynomial as it's the one that looks the most coherent.  

## Raw result

In [ ]:
chosen_soh = (
    charging_points
    .groupby("original_vehicle_id", observed=True, as_index=False)
    .agg(
        soh=pd.NamedAgg("soh_4", "median"),
        odometer=pd.NamedAgg("odometer", "last"),
        vehicle_id=pd.NamedAgg("vehicle_id", "last"),
    )
)
px.scatter(
    chosen_soh.astype({"original_vehicle_id": "category"}),
    x="odometer",
    y="soh",
    color="original_vehicle_id",
    title="Evolution of State of Healt(SoH) over odometer",
    labels={"soh": "State of Healt(%)", "odometer": "odometer(km)"}
    # trendline="ols"
)

## Processed result
We do have an outlier in the vheicle_id 27 so we will remove it for the processed result.

In [ ]:
SOH_UP_BOUND = 100
SOH_LOW_BOUND = 99

fig = px.scatter(
    (
        chosen_soh
        .query("vehicle_id != 27")
        .astype({"original_vehicle_id": "category"})
        .eval("min_max_normed_soh = soh.sub(soh.min()).div(soh.max() - soh.min())")
        .eval("soh = min_max_normed_soh.mul(@SOH_UP_BOUND - @SOH_LOW_BOUND) + @SOH_LOW_BOUND")
    ),
    x="odometer",
    y="soh",
    color="original_vehicle_id",
    title="Evolution of State of Healt(SoH) over odometer",
    labels={"soh": "State of Health(%)", "odometer": "Odometer(km)"}
)
fig.write_html("data_cache/plots/SoH_per_vin_over_odometer.html") 
# fig.write_image("data_cache/plots/SoH_per_vin_over_odometer.png") 
fig.show()

## Conclusion
The small amount of data makes it easy for LR model to overfit the dataset.  
The vins are not too far away from each other but I am afraid we still don't have enough vehicles and odometer depth to properly evaluate our estimation.   
Nevertheless, it seems possible to compute the SoH.  